# Define and assign bondwire profiles

## Imports

In [1]:
import os
from ansys.edb.database import Database
from ansys.edb.layout.cell import Cell, CellType
from ansys.edb.hierarchy.via_group import ViaGroup
from ansys.edb.layer import LayerType, StackupLayer, ViaLayer
from ansys.edb.layer.layer_collection import LayerCollection, LayerCollectionMode
from ansys.edb.net import Net
from ansys.edb.session import launch_session
from ansys.edb.hierarchy import CellInstance
from ansys.edb.definition import ApdBondwireDef
from ansys.edb.primitive import Bondwire, BondwireType, Rectangle, RectangleRepresentationType, BondwireCrossSectionType

## Configs

In [2]:
# RPC_SERVER = None if separate server is already running, Root dir of server executable otherwise.
RPC_SERVER = None
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT, "define_and_assign_bondwire_profiles.aedb")

## Launch a session

In [3]:
# Make sure old one has been disconnected prior to starting a new one, if any
session = launch_session(RPC_SERVER, 50051)

## Basic setup of empty EDB

In [4]:
"""Create Database, Cell, Layout, Net"""
db = Database.create(EDB_FILE)
cell = Cell.create(db, CellType.CIRCUIT_CELL, "EMDesign1")
layout = cell.layout

# Cell to create the cell instances
cell1 = Cell.create(db, CellType.CIRCUIT_CELL, "EMDesign2")
cell2 = Cell.create(db, CellType.CIRCUIT_CELL, "EMDesign3")

# Layouts for the additional cells
layout_start = cell1.layout
layout_end = cell2.layout

# Net
net = Net.create(layout, 'GND')
nets = Net.create(layout_start, 'GND')
nete = Net.create(layout_end, 'GND')

id: 32



## Setup Layers for the different cells

In [5]:
"""Create layers, add them to layer collection and retrieve them again."""

# Layers in layout's layer collection
layer_gnd = StackupLayer.create('Ground', LayerType.SIGNAL_LAYER, 0.0001, 0, "copper")
layer_1 = StackupLayer.create('Layer1', LayerType.SIGNAL_LAYER, 0.0001, 0.0001, "copper")
layer_2 = StackupLayer.create('Layer2', LayerType.SIGNAL_LAYER, 0.0001, 0.0002, "copper")
lc = layout.layer_collection
lc.add_layers([layer_gnd, layer_1, layer_2])

# Layers for additional cells
lcs = layout_start.layer_collection
lcs.add_layers([layer_1])
lce = layout_end.layer_collection
lce.add_layers([layer_2])

# Retrieve layers collection and layers
layout.layer_collection = lc
layer_gnd = lc.find_by_name('Ground')
layout_start.layer_collection = lcs
layer_1 = lcs.find_by_name('Layer1')
layout_end.layer_collection = lce
layer_2 = lce.find_by_name('Layer2')

# Priint layers created for lc
print("LC", [(l.name) for l in lc.get_layers() ])
# Priint layers created for lcs
print("LCS", [(l.name) for l in lcs.get_layers() ])
# Priint layers created for lcs
print("LCE", [(l.name) for l in lce.get_layers() ])

LC ['Layer2', 'Layer1', 'Ground']
LCS ['Layer1']
LCE ['Layer2']


## Create geometry

In [6]:
"""Create Primitive objects"""

# Rectangles
um = lambda val : val * 1e-6
rect = Rectangle.create(layout=layout,
                        layer=layer_gnd,
                        net=net,
                        rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                        param1=um(10),
                        param2=um(10),
                        param3=um(20),
                        param4=um(20),
                        corner_rad=0.0,
                        rotation=0.0
                        )

rect_2 = Rectangle.create(layout=layout,
                          layer=layer_gnd,
                          net=net,
                          rep_type=RectangleRepresentationType.LOWER_LEFT_UPPER_RIGHT,
                          param1=um(0.0),
                          param2=um(0.0),
                          param3=um(30),
                          param4=um(30),
                          corner_rad=0.0,
                          rotation=0.0
                          )
#Retrieve the primitives
primitives_created = layout.primitives

# Print primitives created and their layers
print([(p.primitive_type.name, p.layer.name) for p in primitives_created ])

[('RECTANGLE', 'Ground'), ('RECTANGLE', 'Ground')]


## Create the cell instances

In [7]:
cis = CellInstance.create(layout, 'CellInstanceStart', layout_start)
cie = CellInstance.create(layout, 'CellInstanceEnd', layout_end)

## Create the bondwire definition

In [8]:
# Create an APD Bondwire Definition
wire_0 = ApdBondwireDef.create(db, 'Wire_0')

# Bondwire Parameters name (nm) diameter(dia) and material(mat) should be set as strings(quoted)
parameters_str = "bwd(nm=\'Wire_0\', ven=true, for=true, dia=\'2.54e-05\', mat=\'GOLD\', col=0, vis=true, dih=0, nfc=6, seg(ht=0, hv=0, vt=0, vv=0.000254, ht=1, hv=0.125, vt=0, vv=0))"

# Set parameters
wire_0.set_parameters(parameters_str)

## Create the bondwire

In [9]:
# Bondwire
bondwire = Bondwire.create(
    layout=layout,
    bondwire_type=BondwireType.APD,
    definition_name='Wire_0',
    placement_layer=layer_gnd.name,
    width=1e-3,
    material='GOLD',
    start_context=cis,
    start_layer_name=layer_1.name,
    start_x=0.0,
    start_y=0.0,
    end_context=cie,
    end_layer_name=layer_2.name,
    end_x=1.2,
    end_y=3.5,
    net=net,
)
bondwire.cross_section_type = BondwireCrossSectionType.RECTANGLE
bondwire.cross_section_height = 0.1
bondwire.set_traj(0.1, 0.2, 0.3, 0.4)

# Save database

In [10]:
"""Save database if you wish."""

db.save()

# Disconnect from database

In [11]:
"""Don't forget to disconnect it!"""

session.disconnect()